In [1]:
!pip install -q -U transformers accelerate datasets bitsandbytes peft trl datasets

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModelForCausalLM,TrainingArguments,BitsAndBytesConfig,pipeline,Trainer,DataCollatorForLanguageModeling
import torch
from peft import LoraConfig,PeftModel,prepare_model_for_kbit_training,get_peft_model,TaskType
from trl import SFTTrainer
from warnings import filterwarnings
filterwarnings("ignore")

2024-05-19 04:59:38.551197: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 04:59:38.551296: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 04:59:38.655626: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
dataset = load_dataset("b-mc2/sql-create-context")

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['answer', 'question', 'context'],
        num_rows: 78577
    })
})

In [5]:
base_model_id= "NousResearch/Hermes-2-Pro-Mistral-7B"

In [6]:
tokenizer= AutoTokenizer.from_pretrained(base_model_id,use_fast=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side='right'

tokenizer_config.json:   0%|          | 0.00/6.86k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
def tokenize_data(sample):
    start_prompt= "Write SQL query according to given question or instruction.\n\n"
    end_prompt= "\n\nResponse:"
    prompt= [start_prompt+question+end_prompt for question in sample['question']]
    sample['input_ids']= tokenizer(prompt,padding='max_length',truncation=True,return_tensors='pt',max_length=512,).input_ids
    sample['labels']= tokenizer(sample['answer'],padding='max_length',truncation=True,return_tensors='pt',max_length=512).input_ids
    
    return sample

In [8]:
dataset= dataset.map(tokenize_data,batched=True)

Map:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [9]:
dataset=dataset.remove_columns(['answer', 'question', 'context'])

In [10]:
dataset=dataset['train']

In [11]:
dataset= dataset.select(range(1000))

In [12]:
# Quantization Config 
bnb_config= BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True)

# LoRA COnfig
peft_config= LoraConfig(lora_alpha=15,
                        lora_dropout=0.1,
                        target_modules=[
                                    'q_proj',
                                    'k_proj',
                                    'v_proj',
                                    'dense'],
                        bias="none",
                        task_type=TaskType.CAUSAL_LM)

In [13]:
base_model= AutoModelForCausalLM.from_pretrained(base_model_id,quantization_config=bnb_config,device_map="auto")
base_model.config.use_cache=False

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.68G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [14]:
quatnized_model= prepare_model_for_kbit_training(base_model)

In [20]:
peft_model= get_peft_model(model=quatnized_model,peft_config=peft_config)

In [16]:
merged_model = peft_model.merge_and_unload()

In [17]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

In [18]:
from huggingface_hub import notebook_login
notebook_login()

In [21]:
training_args= TrainingArguments(
    output_dir='/kaggle/working/Mistral-7B-Fine_Tunned',
    hub_model_id='Shorya22/Mistral-7B-Fine_Tunned',
    learning_rate=2e-4,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    logging_steps=10,
    save_strategy='epoch',
    save_steps=10)

trainer= Trainer(
    model= peft_model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
    data_collator=data_collator)

In [22]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,1.381700,1.313442


TrainOutput(global_step=125, training_loss=1.5632500839233399, metrics={'train_runtime': 3937.5112, 'train_samples_per_second': 0.254, 'train_steps_per_second': 0.032, 'total_flos': 2.185884598272e+16, 'train_loss': 1.5632500839233399, 'epoch': 1.0})

In [23]:
new_model= "Mistral-7B-Fine_Tunned1"

In [24]:
trainer.model.save_pretrained(new_model)

In [25]:
new_model= '/kaggle/working/Mistral-7B-Fine_Tunned1'

In [28]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Define the base model ID or path
base_model = 'NousResearch/Hermes-2-Pro-Mistral-7B'  # replace with actual path or model ID

# Load the base model with appropriate parameters
model = AutoModelForCausalLM.from_pretrained(base_model,
                                             low_cpu_mem_usage=True,
                                             return_dict=True,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto")

# Define the path or model ID for the QLoRA adapters
lora_adapters = '/kaggle/working/Mistral-7B-Fine_Tunned1'  # replace with actual path or model ID

# Load the QLoRA adapters into the model
model = PeftModel.from_pretrained(model, lora_adapters)

# Merge the QLoRA adapters into the base model and unload adapters
model = model.merge_and_unload()

# Load the tokenizer for the base model
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side='right'

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [30]:
model.push_to_hub("Mistral-7B-Fine_Tunned_Model")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Shorya22/Mistral-7B-Fine_Tunned_Model/commit/8f93a8f01e08022b500725aaef7687b52a2d0065', commit_message='Upload MistralForCausalLM', commit_description='', oid='8f93a8f01e08022b500725aaef7687b52a2d0065', pr_url=None, pr_revision=None, pr_num=None)

In [29]:
tokenizer.push_to_hub("Mistral-7B-Fine_Tunned_Model")

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Shorya22/Mistral-7B-Fine_Tunned_Model/commit/d40be50fa941510c2ce9743560f37f95804ce9b3', commit_message='Upload tokenizer', commit_description='', oid='d40be50fa941510c2ce9743560f37f95804ce9b3', pr_url=None, pr_revision=None, pr_num=None)

In [31]:
ll= '''You have a database with two tables: employees and departments. The employees table contains information about each employee, and the departments table contains information about each department.Write an SQL query to retrieve the names of all employees along with the name of the department they belong to.'''

In [32]:
start_prompt= "Write SQL query according to given question or instruction.\n\n"
end_prompt= "\n\nResponse:"
prompt= start_prompt+ll+end_prompt

pipe= pipeline(task='text-generation',model=model,tokenizer=tokenizer,max_length=200)
result= pipe(prompt)
print(result[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Write SQL query according to given question or instruction.

You have a database with two tables: employees and departments. The employees table contains information about each employee, and the departments table contains information about each department.Write an SQL query to retrieve the names of all employees along with the name of the department they belong to.

Response:

SELECT employees.name, departments.name
FROM employees, departments
WHERE employees.department_id = departments.department_id;

Explanation:
The query is asking to retrieve the names of all employees along with the name of the department they belong to. To achieve this, we use a SELECT statement to select the employees' name and department's name, followed by a SQL query with a WHERE clause to join the two tables (employees and departments) using the common 'department_id' column. This query returns the desired information.
